Evaluation for BERT

Load the validation datasets

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime
import pickle

In [ ]:
import sys
 
# setting path for common utils script
sys.path.append('../')

import str_cleaning_functions
import dataset_loader

import evaluation_functions

In [ ]:
DATASET_SIZE = 120
DATASET_IS_BALANCED = True

training_name = 'bert-finetune_{}k_{}'.format(
    DATASET_SIZE,
    'bal' if DATASET_IS_BALANCED else 'imbal'
)

training_args_datetime = datetime(year=2023, month=12, day=17)
training_storing_folder = Path(training_name).resolve()

dataset_traintest, X_imbal_valid, y_imbal_valid, X_bal_valid, y_bal_valid = dataset_loader.load_presampled_dataset(DATASET_SIZE, DATASET_IS_BALANCED)

X_train, X_test, y_train, y_test = dataset_loader.create_traintest_dataset(dataset_traintest)

In [ ]:
# data cleaning function

def cleaning_arr(str_arr):
    str_arr = str_arr.apply(lambda x: str_cleaning_functions.clean(x))
    str_arr = str_arr.apply(lambda x: str_cleaning_functions.deEmojify(x))

    return str_arr

In [ ]:
X_train = cleaning_arr(X_train)
X_test = cleaning_arr(X_test)

X_imbal_valid = cleaning_arr(X_imbal_valid)
X_bal_valid = cleaning_arr(X_bal_valid)

In [ ]:
print('Validation (imbalanced) set')
print(len(X_imbal_valid))
print(len(y_imbal_valid))
print()
print('Validation (balanced) set')
print(len(X_bal_valid))
print(len(y_bal_valid))

print()
print()

print('Validation (imbalanced) set')
print(pd.Series(y_imbal_valid).value_counts())
print()
print('Validation (balanced) set')
print(pd.Series(y_bal_valid).value_counts())

print()
print()

print('validation (imbalanced) set datatype')
print(type(X_imbal_valid))
print(type(y_imbal_valid))
print()
print('validation (balanced) set datatype')
print(type(X_bal_valid))
print(type(y_bal_valid))

Load the models

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained(
    Path.joinpath(
        training_storing_folder, 
        '{}_model'.format(training_args_datetime.strftime('%Y-%m-%d')))
)

pipeline = pipeline(
    'text_classification',
    model=model,
    tokenizer=tokenizer
)

Evaluation

In [ ]:
pipeline.predict(X_test)